In [4]:
from IPython.display import Image

## trainer arguments & trainer

In [1]:
from transformers import TrainingArguments, Trainer
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class DistillTrainingArguments(TrainingArguments):
    # 增加两个 KD 所需的参数参数
    # TrainingArguments: @dataclass
    def __init__(self, *args, alpha=0.5, temperature=2., **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

In [3]:
class DistillTrainer(Trainer):
    pass